# Local-in-time rerun

In [ ]:
%load_ext autoreload
import tools
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import copy
import time
%load_ext snakeviz
import os   
import pickle
import pprint
from lit import *
%autoreload 2



# Plotting and post-processing

In [ ]:



plt.semilogy(hist_time,np.array(hist_th_hm1),label= 'H^-1')
plt.semilogy(hist_time,np.array(hist_th_l2),label= 'L^2')
plt.semilogy(hist_time,np.array(hist_th_h1),label= 'H^1')
hm1= hist_hm1[0]
h1 = st.h1norm(hist_th[0])
l2 = hist_l2[0]
t = time_array
pred = hm1*np.exp(-t-1/(2.*Pe)*h1**2./l2**2.*(np.exp(2.*t)-1.))
#pred = hm1*np.exp(-t)
plt.semilogy(time_array, pred, label= 'Lower Bound')
plt.legend()
#plt.xlim([0,0.5])
plt.ylim([1e-6,1e2])
plt.show()


lam = (2.*np.pi*np.array(hist_th_hm1)/np.array(hist_th_l2))
plt.semilogy(hist_time,lam,label= r'$\lambda$')
plt.legend()
#plt.xlim([0,0.5])
#plt.ylim([1e-6,1e2])
plt.show()

plt.semilogy(hist_time,(2.*np.pi)**-0.5*lam/(1./(Pe)**0.5),label= r'$\lambda / \lambda_B$')
plt.legend()
#plt.xlim([0,0.5])
#plt.ylim([1e-6,1e2])
plt.show()

plt.semilogy(hist_time,np.array(hist_u_h1),label= 'velocity H^1')
plt.semilogy(hist_time,np.array(hist_u_l2),label= 'velocity L^2')
#plt.semilogy(hist_time,[np.amax(np.array(u)) for u in hist_u],label= 'velocity L^\infty')
plt.legend()
#plt.xlim([0,0.5])
#plt.ylim([1e-6,1e2])
plt.show()


r= np.zeros(len(hist_time))
for i in range(len(hist_time)):
    if i == 0 :
        delta_t = hist_time[i+1]-hist_time[i]
        dhm1_dt = (hist_th_hm1[i+1]-hist_th_hm1[i])/delta_t
        r[i] = dhm1_dt/hist_th_hm1[i]
        
    if i > 0 and i < len(hist_time)-1:
        delta_t = hist_time[i+1]-hist_time[i-1]
        dhm1_dt = (hist_th_hm1[i+1]-hist_th_hm1[i-1])/delta_t
        r[i] = dhm1_dt/hist_th_hm1[i]
        
    if i == len(hist_time)-1:
        delta_t = hist_time[i]-hist_time[i-1]
        dhm1_dt = (hist_th_hm1[i]-hist_th_hm1[i-1])/delta_t
        r[i] = dhm1_dt/hist_th_hm1[i]
        
plt.semilogy(hist_time,-r,label= 'r(t)')
plt.legend()
plt.show()

# Testing Convergence

In [ ]:
s = []
ss=[]
N= 256
L = 1.0
M_list = [8,16,32,64] 
T = 0.01
Pe = 512

dt_list = []
for M in M_list:
    s.append(sim(N = N, M = M, T =T, L = L,save_every =1,
                 T_kick=0.01, Pe = Pe, plot=False))
    dt_list.append(T/M)
error_list = []
st = tools.ScalarTool(N,L)
for i in range(len(M_list)-1):
        error_list.append(st.l2norm(s[i].hist_th[-1] - s[-1].hist_th[-1]))

error_array = np.array(error_list)
dt_array = np.array(dt_list)
print(st.l2norm(s[-3].hist_th[-1] - s[-2].hist_th[-1]))
print(st.l2norm(s[-2].hist_th[-1] - s[-1].hist_th[-1]))
R = st.l2norm(s[-3].hist_th[-1] - s[-2].hist_th[-1])/st.l2norm(s[-2].hist_th[-1] - s[-1].hist_th[-1])
p = np.log(R)/np.log(2)
print('p = ', p)

In [ ]:
plt.loglog(dt_array[:-1], error_array, label = 'error')
plt.loglog(dt_array, error_array[0]*(dt_array/dt_array[0])**4, label = 'expected')
plt.legend()
plt.show()
print(error_array)
print(dt_array)
print([s[i].hist_time[-1] for i in range(len(s))])

In [ ]:
st.l2norm(s[2].hist_th[0] - s[1].hist_th[1])

In [ ]:
gamma = 1.0
N = 128
L= 1.0
kappa = 0.0
st = tools.ScalarTool(N,L)
vt = tools.VectorTool(N,L)
ftype = np.float64
T=0.32
M_list=[32,64,128,256,512]


def f(th,u):
    return -1.0*np.sum(vt.dealias(u)*st.grad(st.dealias(th)),0) + kappa * st.lap(st.dealias(th))
    #return -1.0*np.sum(u*st.grad(th),0) + kappa * st.lap(th)
def u_lit(th0):
    u_lit = st.dealias(th0) * vt.dealias(st.grad_invlap(th0))
    #print(vt.l2norm(u_lit))
    u_lit = - vt.invlap(vt.div_free_proj(u_lit))
    u_lit = gamma * L * u_lit / st.l2norm(vt.curl(u_lit))
    return u_lit

# def f_lit(th):
#     u_kick = np.zeros((2,N,N),dtype = ftype)
#     u_kick[0,:,:] = np.sin(2.*np.pi*X[1]/L)
#     return f(th,u_kick)

def f_lit(th):
    return f(th,u_lit(th))


for dt_kick in [0.1,0.01,0.001,0.0001,0.0000000001]:

    th_list = []
    th =np.sin(2*np.pi*st.X[0]/L)
    u_kick = np.zeros((2,N,N),dtype = ftype)
    u_kick[0,:,:] = np.sin(2.*np.pi*st.X[1]/L)
    th_kick= th + dt_kick*f(th,u_kick)

    dt_list = []
    for M in M_list:
        dt = T/M
        th =copy.copy(th_kick)
        for i in range(M):
            k1 = f_lit(th)
            k2 = f_lit(th + 0.5*dt*k1)
            k3 = f_lit(th + 0.5*dt*k2)
            k4 = f_lit(th + dt*k3)
            th = th + dt*(1./6.)*(k1 + 2.*k2 + 2.*k3 + k4)
            #th = th + dt*k1
        th_list.append(th)
        dt_list.append(dt)


    error_list = []
    for i in range(0,len(M_list)-1):
        error_list.append(st.l2norm(th_list[i]-th_list[-1]))
    plt.figure()
    plt.loglog(dt_list[:-1],error_list)
    plt.loglog(dt_list,np.array(dt_list)**4)
    #plt.ylim(1e0,1e2)
    plt.show()

In [ ]:
dt_kick = 0.001
th =np.sin(2*np.pi*st.X[0]/L)
u_kick = np.zeros((2,N,N),dtype = ftype)
u_kick[0,:,:] = np.sin(2.*np.pi*X[1]/L)


th= th + dt_kick*f(th,u_kick)

u_lit = st.dealias(th) * vt.dealias(st.grad_invlap(th))
print(vt.l2norm(u_lit))
u_lit = - vt.invlap(vt.div_free_proj(u_lit))
print(vt.l2norm(u_lit))
u_lit = gamma * L * u_lit / st.l2norm(vt.curl(u_lit))
print(vt.h1norm(u_lit))   

In [ ]:
vt.l2norm(u1-u0)

In [ ]:
pickle_file = "/Volumes/Transcend/LIT/output-pe=1024/pe=1024.pkl"

with open(pickle_file, 'rb') as f:
    d = pickle.load(f, encoding='latin1') 


In [ ]:
time_1024,th_1024 = d

In [ ]:
N=256
L=1.0
st = tools.ScalarTool(N,L)
st.plot(th_1024[-1])

In [ ]:
N=256
L=1.0
Pe = 1024
T=time_1024[-1]
dt = 0.001
sim_1024 = sim(N = N, dt = dt, T =T, L = L, Pe = Pe, save_every = 100, plot=True)

In [ ]:
st.plot(th_1024[-1])
plt.show()
st.plot(sim_1024.hist_th[-1])
plt.show()

In [ ]:
st.hm1norm(th_1024[-1])

In [ ]:
st.hm1norm(sim_1024.hist_th[-1])

In [ ]:
## time_1024[-1]2

In [ ]:
plt.semilogy(sim_1024.hist_time,np.array(sim_1024.hist_th_hm1))
plt.semilogy(time_1024, [ st.hm1norm(th) for th in th_1024 ])
plt.show()

In [ ]:
plt.semilogy(sim_1024.hist_time,np.array(sim_1024.hist_th_hm1))
plt.semilogy(time_1024, [ st.hm1norm(th) for th in th_1024 ])
plt.show()

In [ ]:
pickle_file = "output-pe=128.0-M=26/pe=128.0.pkl"

with open(pickle_file, 'rb') as f:
    d = pickle.load(f, encoding='latin1') 

In [ ]:
d.hist_time

In [ ]:
sim(constraint = "energy",T=0.3,M=1000,N=64,plot=True,Pe=32)

# Test convergence for fixed Pe and series of M values

In [ ]:
M_list = [82,164,328] 
Pe = 128.0
s = []
for M in M_list:
    pickle_file = "output-pe="+str(Pe)+"-M="+str(M)+"/pe="+str(Pe)+"-M="+str(M) +".pkl"

    with open(pickle_file, 'rb') as f:
        s.append(pickle.load(f, encoding='latin1'))
    

N = s[0].N
L = s[0].L
dt_list = [item.T/item.M for item in s]
error_list = []
st = tools.ScalarTool(N,L)
for i in range(len(M_list)-1):
        error_list.append(st.l2norm(s[i].hist_th[-1] - s[-1].hist_th[-1]))

error_array = np.array(error_list)
dt_array = np.array(dt_list)
print(st.l2norm(s[-3].hist_th[-1] - s[-2].hist_th[-1]))
print(st.l2norm(s[-2].hist_th[-1] - s[-1].hist_th[-1]))
R = st.l2norm(s[-3].hist_th[-1] - s[-2].hist_th[-1])/st.l2norm(s[-2].hist_th[-1] - s[-1].hist_th[-1])
p = np.log(R)/np.log(2)
print('p = ', p)

In [ ]:
s
dt_list

In [ ]:
Pe_dict = {}

In [ ]:
Pe_dict[128] = [100,200,300]
Pe_dict[64] = [50,300,600]

In [ ]:
Pe_dict

In [ ]:
f = open('report.txt', 'w')
f.write('a line of text \n')
f.write('a line of text \n')
f.close()

In [ ]:
M = 164
Pe = 128.0


pickle_file = "output-pe="+str(Pe)+"-M="+str(M)+"/pe="+str(Pe)+"-M="+str(M) +".pkl"

with open(pickle_file, 'rb') as f:
    s = pickle.load(f, encoding='latin1')
    
